In [5]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import mediapipe as mp
from pathlib import Path  # for safer path joins if needed
import pandas as pd

# access utils if using shared code
import sys
sys.path.append("..")

In [6]:
def roi_crop(input_path, output_path, label):

    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True, 
    refine_landmarks=True, 
    max_num_faces=1, 
    min_detection_confidence=0.5 # inc if too many fase detections
)
    LEFT_EYE_LANDMARKS = [463, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382, 362]
    RIGHT_EYE_LANDMARKS = [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7]

    # iris not needed for conjunctivitis
    LEFT_IRIS_LANDMARKS = [474, 475, 477, 476]
    RIGHT_IRIS_LANDMARKS = [469, 470, 471, 472]

    skipped_files = []

    for file in os.listdir(input_path):
        img_path = os.path.join(input_path, file)

        try:
            # loads the image
            img = cv2.imread(img_path)

            if img is None or len(img.shape) != 3 or img.shape[2] != 3:
                raise Exception
            
            # convert img to RGB for mediapipe
            # opencv default is BGR
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            results = face_mesh.process(img_rgb)

            if not results.multi_face_landmarks:
                print(f"No face detected: {file}")
                continue
            
            # get actual positions on img
            h, w, _ = img.shape
            landmarks = results.multi_face_landmarks[0]
            eye_points = []
            for idx in LEFT_EYE_LANDMARKS + RIGHT_EYE_LANDMARKS:
                x = int(landmarks.landmark[idx].x * w)  # scale x to image width
                y = int(landmarks.landmark[idx].y * h)  # scale y to image height
                eye_points.append((x, y))
            
            x_coords, y_coords = zip(*eye_points)  # separates xs and ys

            # find min and max x and y for bounding rectangle
            x_min, x_max = max(min(x_coords), 0), min(max(x_coords), w)
            y_min, y_max = max(min(y_coords), 0), min(max(y_coords), h)

            if x_max - x_min < 10 or y_max - y_min < 10:
                skipped_files.append((file, "ROI too small to crop"))
                continue

            roi = img[y_min:y_max, x_min:x_max]  # crop using the box
            roi_resized = cv2.resize(roi, (224, 224))  # resize to model input size

            save_path = os.path.join(output_path, file)
            cv2.imwrite(save_path, roi_resized)

        except Exception as e:
            skipped_files.append((file, f"Processing error: {e}"))
    return skipped_files

In [7]:
processed_healthy_path = '../data/processed/healthy_eye'
processed_infected_path = '../data/processed/infected_eye'

roi_healthy_path = '../data/roi/healthy_eye'
roi_infected_path = '../data/roi/infected_eye'

# Run the cropper
skipped_healthy_roi = roi_crop(processed_healthy_path, roi_healthy_path, "Healthy")
skipped_infected_roi = roi_crop(processed_infected_path, roi_infected_path, "Infected")

# Combine skipped logs
skipped_roi = skipped_healthy_roi + skipped_infected_roi

I0000 00:00:1750372900.241284 21125262 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Max
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1750372900.244047 22039203 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750372900.250410 22039199 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750372900.265878 22039201 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


No face detected: 63.jpg
No face detected: 77.jpg
No face detected: 162.jpg
No face detected: 88.jpg
No face detected: 172.jpeg
No face detected: 125.jpeg
No face detected: 89.jpg
No face detected: 163.jpg
No face detected: 62.jpg
No face detected: 74.jpg
No face detected: 60.jpg
No face detected: 48.jpg
No face detected: 149.jpg
No face detected: 161.jpg
No face detected: 129.jpeg
No face detected: 160.jpg
No face detected: 148.jpg
No face detected: 49.jpg
No face detected: 61.jpg
No face detected: 75.jpg
No face detected: 59.jpg
No face detected: 65.jpg
No face detected: 164.jpg
No face detected: 159.jpg
No face detected: 165.jpg
No face detected: 64.jpg
No face detected: 8.jpg
No face detected: 66.jpg
No face detected: 72.jpg
No face detected: 167.jpg
No face detected: 99.jpg
No face detected: 132.jpeg
No face detected: 166.jpg
No face detected: 98.jpg
No face detected: 73.jpg
No face detected: 67.jpg
No face detected: 9.jpg
No face detected: 14.jpg
No face detected: 101.jpg
No face

I0000 00:00:1750372900.831340 21125262 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Max
W0000 00:00:1750372900.832579 22039283 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750372900.837502 22039283 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


No face detected: 172.jpeg
No face detected: 89.jpg
No face detected: 164.jpeg
No face detected: 62.jpg
No face detected: 74.jpg
No face detected: 60.jpg
No face detected: 48.jpg
No face detected: 149.jpg
No face detected: 175.jpg
No face detected: 168.jpeg
No face detected: 148.jpg
No face detected: 49.jpg
No face detected: 75.jpg
No face detected: 59.jpg
No face detected: 71.jpg
No face detected: 65.jpg
No face detected: 169.jpeg
No face detected: 153.jpeg
No face detected: 64.jpg
No face detected: 70.jpg
No face detected: 58.jpg
No face detected: 8.jpg
No face detected: 66.jpg
No face detected: 165.jpeg
No face detected: 72.jpg
No face detected: 99.jpg
No face detected: 173.jpeg
No face detected: 98.jpg
No face detected: 73.jpg
No face detected: 67.jpg
No face detected: 9.jpg
No face detected: 14.jpg
No face detected: 28.jpg
No face detected: 129.jpg
No face detected: 101.jpg
No face detected: 115.jpg
No face detected: 100.jpg
No face detected: 17.jpg
No face detected: 102.png
No fa

In [8]:
import pandas as pd

if skipped_roi:
    df_skipped = pd.DataFrame(skipped_roi, columns=["filename", "reason"])
    df_skipped.to_csv("../data/log/skipped_roi_images.csv", index=False)
    print(f"Logged {len(skipped_roi)} skipped files to '../data/log/skipped_roi_images.csv'")
else:
    print("All ROI crops successful.")

Logged 1 skipped files to '../data/log/skipped_roi_images.csv'
